Usamos los datos de la JODI Oil World Database (Joint Organisations Data Initiative [JODI], 2018). Que se obtiene de aqui: https://www.jodidata.org/oil/database/data-downloads.aspx

Fecha de descarga: 1 septiembre de 2020





In [3]:
import pandas as pd
monthly_first = pd.read_csv("../Databases/JODI DB/world_Primary_CSV_NewFormat.csv")
monthly_second = pd.read_csv("../Databases/JODI DB/world_Secondary_CSV_NewFormat.csv")

monthly_first = pd.DataFrame(monthly_first)
monthly_second = pd.DataFrame(monthly_second)

# As the DB has country names referd by alpha-2-code I need to use pycountry to get actual names
#pip install pycountry
import pycountry

In [41]:


# List of countries we are interested in
# Netos = ['Albania','Angola','Saudi Arabia','Algeria','Argentina','Benin','Azerbaijan','Bolivia (Plur. State of)','Brazil','Brunei Darussalam','Cameroon','Canada','China','Colombia','Congo','Denmark','Ecuador','Egypt','United Arab Emirates','Estonia','Gabon','Indonesia','Georgia','Ghana','Guatemala','Iran (Islamic Rep. of)','Iraq','Kazakhstan','Kyrgyzstan','Kuwait','Libya','Malaysia','Mexico','Mongolia','Myanmar','Nigeria','Norway','Oman','Qatar','United Kingdom','Dem. Rep. of the Congo','Russian Federation','Trinidad and Tobago','Sudan','South Sudan','Tunisia','Turkmenistan','Viet Nam','Venezuela (Bolivar. Rep.)','Yemen']

#While passing the code below we find that names from countries as Bolivia, Iran, DR Congo and Venezuela had different names so we have to manually addapt the names to "Bolivia, Plurinational State of", "Iran, Islamic Republic of", 'Congo, The Democratic Republic of the' and "Venezuela, Bolivarian Republic of"
netos = ['Albania','Angola','Saudi Arabia','Algeria','Argentina','Benin','Azerbaijan','Bolivia, Plurinational State of','Brazil','Brunei Darussalam','Cameroon','Canada','China','Colombia','Congo','Denmark','Ecuador','Egypt','United Arab Emirates','Estonia','Gabon','Indonesia','Georgia','Ghana','Guatemala','Iran, Islamic Republic of','Iraq','Kazakhstan','Kyrgyzstan','Kuwait','Libya','Malaysia','Mexico','Mongolia','Myanmar','Nigeria','Norway','Oman','Qatar','United Kingdom','Congo, The Democratic Republic of the','Russian Federation','Trinidad and Tobago','Sudan','South Sudan','Tunisia','Turkmenistan','Viet Nam','Venezuela, Bolivarian Republic of','Yemen']

#Create the same list with the alpha_2 code
a_codes = []
for x in netos:
    try:
        country = pycountry.countries.get(name=x)
        a_codes.append(country.alpha_2)
    except AttributeError:
        print(f'Theres an error with {x}')
        pass

#Check lenght
len(netos) - len(a_codes)

0

In [ ]:
available = monthly_first["REF_AREA"].unique().tolist()

for x in a_codes:
    if x not in available:
        country = pycountry.countries.get(alpha_2=x)
        print(country.name)
#On this data set there are no data for this countries:
# Benin
# Cameroon
# Congo
# Ghana
# Kyrgyzstan
# Mongolia
# Congo, The Democratic Republic of the
# South Sudan
# Turkmenistan


In [26]:
#Filter the data base: show data only on Thousand Barrels per day (kb/d)(KBD)
dayly_barrels = monthly_first.loc[monthly_first["UNIT_MEASURE"] == "KBD"]
#Conversion factor barrels/ktons 
ktons = monthly_first.loc[monthly_first["UNIT_MEASURE"] == "CONVBBL"]

#Filter the data base: show data only from countries we are interested in
exportadores = monthly_first.loc[monthly_first["REF_AREA"].isin(a_codes)]



In [28]:
#divide the data set in three areas of interest crude oil, ngl and total
crude_oil = exportadores.loc[exportadores["ENERGY_PRODUCT"] == "CRUDEOIL"]
NGL = exportadores.loc[exportadores["ENERGY_PRODUCT"] == "NGL"]
total_crude = exportadores.loc[exportadores["ENERGY_PRODUCT"] == "TOTCRUDE"]

,REF_AREA,TIME_PERIOD,ENERGY_PRODUCT,FLOW_BREAKDOWN,UNIT_MEASURE,OBS_VALUE,ASSESSMENT_CODE
0,AE,2002-01,CRUDEOIL,CLOSTLV,CONVBBL,7596.0,3
1,AE,2002-01,CRUDEOIL,CLOSTLV,KBBL,NaN,3
2,AE,2002-01,CRUDEOIL,CLOSTLV,KBD,NaN,3
3,AE,2002-01,CRUDEOIL,CLOSTLV,KL,NaN,3
4,AE,2002-01,CRUDEOIL,CLOSTLV,KTONS,NaN,3
...,...,...,...,...,...,...,...
11095,AE,2020-06,CRUDEOIL,TRANSBAK,CONVBBL,7596.0,3
11096,AE,2020-06,CRUDEOIL,TRANSBAK,KBBL,0.0,3
11097,AE,2020-06,CRUDEOIL,TRANSBAK,KBD,0.0,3
11098,AE,2020-06,CRUDEOIL,TRANSBAK,KL,0.0,3


,REF_AREA,TIME_PERIOD,ENERGY_PRODUCT,FLOW_BREAKDOWN,UNIT_MEASURE,OBS_VALUE,ASSESSMENT_CODE
0,AE,2002-01,CRUDEOIL,CLOSTLV,CONVBBL,7596.0,3
1,AE,2002-01,CRUDEOIL,CLOSTLV,KBBL,NaN,3
2,AE,2002-01,CRUDEOIL,CLOSTLV,KBD,NaN,3
3,AE,2002-01,CRUDEOIL,CLOSTLV,KL,NaN,3
4,AE,2002-01,CRUDEOIL,CLOSTLV,KTONS,NaN,3
